## 0. Imports

In [1]:
import pandas as pd
import requests 
import bs4   # !pip install beautifulsoup4
import re

## 1.Population by region and year

Information coming from INE (https://www.ine.es/jaxiT3/Datos.htm?t=2915)

In [5]:
df_population=pd.read_csv('../data/inhabitants.csv',sep=';' , encoding='latin-1')
df_population.head()

,Comunidades y Ciudades Autónomas,Tamaño de los municipios,Periodo,Total
0,Total,Total,2021,47.385.107
1,Total,Total,2020,47.450.795
2,Total,Total,2019,47.026.208
3,Total,Total,2018,46.722.980
4,Total,Total,2017,46.572.132


In [6]:
#cogiendo datos a partir de 2015
df_population=df_population[df_population['Periodo']>=2015]         

In [7]:
df_population['Comunidades y Ciudades Autónomas']=df_population['Comunidades y Ciudades Autónomas'].str[3:]

In [8]:
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['al'],'Total')
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['Asturias, Principado de'],'Asturias')
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['Castilla - La Mancha'],'Castilla-La Mancha')
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['Comunitat Valenciana'],'Comunidad Valenciana')
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['Comunitat Valenciana'],'Comunidad Valenciana')
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['Madrid, Comunidad de'],'Madrid')
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['Murcia, Región de'],'Murcia')
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['Navarra, Comunidad Foral de'],'Navarra')
df_population['Comunidades y Ciudades Autónomas'] = \
df_population['Comunidades y Ciudades Autónomas'].replace(['Rioja, La'],'La Rioja')

In [9]:
df_population['Comunidades y Ciudades Autónomas'].unique()

array(['Total', 'Andalucía', 'Aragón', 'Asturias', 'Balears, Illes',
       'Canarias', 'Cantabria', 'Castilla y León', 'Castilla-La Mancha',
       'Cataluña', 'Comunidad Valenciana', 'Extremadura', 'Galicia',
       'Madrid', 'Murcia', 'Navarra', 'País Vasco', 'La Rioja', 'Ceuta',
       'Melilla'], dtype=object)

In [10]:
to_exclude = ['Total','Balears, Illes','Canarias','Ceuta','Melilla']

In [11]:
df_population=df_population[~df_population['Comunidades y Ciudades Autónomas'].isin(to_exclude)]

In [12]:
df_population.rename(columns={'Comunidades y Ciudades Autónomas': 'Region', 
                               'Periodo': 'Year', 'Total': 'Inhabitants'}, inplace=True)

In [13]:
df_population = df_population.drop('Tamaño de los municipios', 1)

/Users/malva/miniconda3/envs/final_project/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [14]:
df_population.head()

,Region,Year,Inhabitants
26,Andalucía,2021,8.472.407
27,Andalucía,2020,8.464.411
28,Andalucía,2019,8.414.240
29,Andalucía,2018,8.384.408
30,Andalucía,2017,8.379.820


In [15]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 26 to 448
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Region       105 non-null    object
 1   Year         105 non-null    int64 
 2   Inhabitants  105 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.3+ KB


## 2. Bank holidays by region and year

Information coming from public source: 

https://administracion.gob.es/pag_Home/atencionCiudadana/calendarios/diasInhabiles.html#-b95725c1af8d

In [17]:
df_bank_holidays=pd.read_csv('../data/Dias_festivos.csv', sep=',' , encoding='utf-8')
df_bank_holidays

,Year,Day,Region
0,2015,1/01/2015,Nacional
1,2015,6/01/2015,Nacional
2,2015,28/02/2015,Andalucía
3,2015,19/03/2015,Comunidad Valenciana
4,2015,19/03/2015,Madrid
...,...,...,...
463,2022,26/12/2022,La Rioja
464,2022,26/12/2022,Cataluña
465,2022,26/12/2022,Castilla y León
466,2022,26/12/2022,Madrid


In [23]:
df_bank_holidays_regions=df_bank_holidays[(df_bank_holidays['Region']!='Nacional')]

In [24]:
df_bank_holidays_regions.head()

,Year,Day,Region
2,2015,28/02/2015,Andalucía
3,2015,19/03/2015,Comunidad Valenciana
4,2015,19/03/2015,Madrid
5,2015,19/03/2015,Murcia
6,2015,19/03/2015,Navarra


In [21]:
df_bank_holidays_national=df_bank_holidays[(df_bank_holidays['Region']=='Nacional')]

In [22]:
df_bank_holidays_national.head()

,Year,Day,Region
0,2015,1/01/2015,Nacional
1,2015,6/01/2015,Nacional
22,2015,3/04/2015,Nacional
32,2015,1/05/2015,Nacional
42,2015,15/08/2015,Nacional


In [25]:
df_bank_holidays_national.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 410
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    60 non-null     int64 
 1   Day     60 non-null     object
 2   Region  60 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.9+ KB


## 3. Calendar

In [30]:
import calendar
from datetime import datetime, date

In [31]:
def create_date_table(start='2015-01-01', end='2022-12-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["Day"] = df.Date.dt.day_name()
    df["Week"] = df.Date.dt.weekofyear
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["Year_half"] = (df.Quarter + 1) // 2
    return df

In [32]:
df_calendar=create_date_table()

/Users/malva/miniconda3/envs/final_project/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


In [33]:
df_calendar.head

<bound method NDFrame.head of            Date        Day  Week  Quarter  Year  Year_half
0    2015-01-01   Thursday     1        1  2015          1
1    2015-01-02     Friday     1        1  2015          1
2    2015-01-03   Saturday     1        1  2015          1
3    2015-01-04     Sunday     1        1  2015          1
4    2015-01-05     Monday     2        1  2015          1
...         ...        ...   ...      ...   ...        ...
2917 2022-12-27    Tuesday    52        4  2022          2
2918 2022-12-28  Wednesday    52        4  2022          2
2919 2022-12-29   Thursday    52        4  2022          2
2920 2022-12-30     Friday    52        4  2022          2
2921 2022-12-31   Saturday    52        4  2022          2

[2922 rows x 6 columns]>

In [34]:
def final_decision(row):
    if row == 'Saturday' or row == 'Sunday' :
        return 1
    else:
        return 0

In [35]:
df_calendar['Weekend']=df_calendar['Day'].apply(lambda row: final_decision(row))

In [36]:
df_calendar

,Date,Day,Week,Quarter,Year,Year_half,Weekend
0,2015-01-01,Thursday,1,1,2015,1,0
1,2015-01-02,Friday,1,1,2015,1,0
2,2015-01-03,Saturday,1,1,2015,1,1
3,2015-01-04,Sunday,1,1,2015,1,1
4,2015-01-05,Monday,2,1,2015,1,0
...,...,...,...,...,...,...,...
2917,2022-12-27,Tuesday,52,4,2022,2,0
2918,2022-12-28,Wednesday,52,4,2022,2,0
2919,2022-12-29,Thursday,52,4,2022,2,0
2920,2022-12-30,Friday,52,4,2022,2,0


## Joining data

In [37]:
df_bank_holidays_national

,Year,Day,Region
0,2015,1/01/2015,Nacional
1,2015,6/01/2015,Nacional
22,2015,3/04/2015,Nacional
32,2015,1/05/2015,Nacional
42,2015,15/08/2015,Nacional
48,2015,12/10/2015,Nacional
65,2015,8/12/2015,Nacional
66,2015,25/12/2015,Nacional
68,2016,1/01/2016,Nacional
69,2016,6/01/2016,Nacional


In [38]:
df_calendar=pd.merge(df_calendar, df_bank_holidays_national,on='Day', how='left')

In [39]:
df_calendar.fillna(0, inplace=True)

In [40]:
df_calendar['Region'].unique()

array([0])

https://datosclima.es/Aemet2013/DescargaDatos.html